In [2]:
def download(pdblist_path):
    """Takes a list of line-separated pdb files and downloads short SOCKET outputs from the pdb"""
    import urllib3
    import mmap
    import os
    import time

    pdblist_path = str(pdblist_path)
    with open(pdblist_path, 'r') as pdbs:
        pdblist = pdbs.read().splitlines()
        pdbs.close

    #remove any duplication from list
    pdblist_nodegen = list( dict.fromkeys(pdblist) )
    #iterate through pdbfiles
    for pdbcode in pdblist_nodegen:
        #create directory for database
        if not os.path.isdir(str(pdblist_path[:-4])+"files/"):
            path = os.makedirs(str(pdblist_path[:-4])+"files/", exist_ok=True)
        else:
            path = str(pdblist_path[:-4])+"files/"
        #import http data
        http = urllib3.PoolManager()
        #Are there multiple coiled-coil sections in a single PDB file? Currently cut at 40 sections
        #to avoid ones that contain hundreds of biological units.
        coiled_coil_outputs = []
        coiled_coil_output_number = []
        for ccsection in range(1,40):
            url_string = str("http://coiledcoils.chm.bris.ac.uk/ccplus/files/data/"+pdbcode[1:3]+"/"+pdbcode+"/socket/"+pdbcode+"_"+str(ccsection)+".mmol.70.short.socket")
            r = http.request('GET', url_string, preload_content=False)
            first_line = r.readlines()[0].decode('ascii')
            if first_line[0:4] == 'SOCK':
                coiled_coil_outputs.append(url_string)
                coiled_coil_output_number.append(ccsection)
            else:
                break
        if len(coiled_coil_outputs) == 0:
            output_file = str(path)+str(pdbcode)+".txt"
        #if it fails to get a file try downloading from this url instead
            r = http.request('GET', str("http://coiledcoils.chm.bris.ac.uk/ccplus/files/data/"+pdbcode[1:3]+"/"+pdbcode+"/socket/"+pdbcode+".pdb.70.short.socket"), preload_content=False)
        #write output file
            with open(output_file, 'wb') as out:
                while True:
                    data = r.read()
                    if not data:
                        break
                    out.write(data)
            out.close()


            r.release_conn()
        else:
            for ccsection_file, section_number in zip(coiled_coil_outputs, coiled_coil_output_number):
                output_file = str(path)+str(section_number)+"_"+str(pdbcode)+".txt"
                r = http.request('GET', ccsection_file, preload_content=False)
                with open(output_file, 'wb') as out:
                    while True:
                        data = r.read()
                        if not data:
                            break
                        out.write(data)
                out.close()
                r.release_conn()
    return pdblist_nodegen

In [ ]:
download('./SOCKET_list.txt')